# name-that-building

In [0]:
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os, sys
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import random
buildingCodes = ["AA", "CB", "CT", "DT", "FH","HP","HS", "LB", "MC",
             "ME", "ML", "PA", "RB", "RO", "SA", "TB"]
test = []
training = []
width = 500
height = 500
combinedDataFrame = []
validationData=[]
validationDataFrame = []
testDataFrame = []
# validation , test and training dataframes
for building in buildingCodes:
  try:
    path =  glob("drive/My Drive/All_Images_500x500/" + str(building) + "/*.jpg")
    
    for i in range(15): # randomly select 15 images from each building directory
      validation = []
      test =[]
      randIndex = random.randint(0,len(path)-1)
      test.append(path.pop(randIndex)) # removing the image path once appened to the dataset
      test.append(building)
      testDataFrame.append(test)

      validation.append(path.pop(-randIndex)) # removing the image path once appened to the dataset
      validation.append(building)
      validationDataFrame.append(validation)

    for img in path: 
      data = []
      data.append(img)
      data.append(building)
      combinedDataFrame.append(data) # appending the training dataset

   
  except (IOError, SyntaxError) as e:
    print('Bad file:', img) 
    path.remove(img)
# creating a panda dataframe for test, train and validation
vdf = pd.DataFrame(validationDataFrame ,columns=["Image","Label"])
df = pd.DataFrame(combinedDataFrame,columns=["Image","Label"])
tdf = pd.DataFrame(testDataFrame,columns=["Image","Label"])
print(vdf)
print(df)
print(len(tdf))


                                              Image Label
0    drive/My Drive/All_Images_500x500/AA/285_2.jpg    AA
1    drive/My Drive/All_Images_500x500/AA/548_2.jpg    AA
2    drive/My Drive/All_Images_500x500/AA/958-1.jpg    AA
3    drive/My Drive/All_Images_500x500/AA/607_2.jpg    AA
4    drive/My Drive/All_Images_500x500/AA/182_1.jpg    AA
..                                              ...   ...
235  drive/My Drive/All_Images_500x500/TB/954-1.jpg    TB
236  drive/My Drive/All_Images_500x500/TB/905_3.jpg    TB
237  drive/My Drive/All_Images_500x500/TB/548_5.jpg    TB
238  drive/My Drive/All_Images_500x500/TB/533_1.jpg    TB
239  drive/My Drive/All_Images_500x500/TB/157_1.jpg    TB

[240 rows x 2 columns]
                                               Image Label
0     drive/My Drive/All_Images_500x500/AA/120-4.jpg    AA
1     drive/My Drive/All_Images_500x500/AA/548_5.jpg    AA
2     drive/My Drive/All_Images_500x500/AA/713_7.jpg    AA
3     drive/My Drive/All_Images_500x500/AA/2

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input

WIDTH = 500
HEIGHT = 500
BATCH_SIZE = 50
# pre training the model
# preprocessing
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode='nearest',
    width_shift_range=0.2,
    height_shift_range=0.2) 
                                   
validation_datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode='nearest',
    width_shift_range=0.2,
    height_shift_range=0.2) 
# training set
train_generator = train_datagen.flow_from_dataframe(df,x_col= "Image", y_col = "Label", classes = buildingCodes, target_size=(500,500), batch_size = 50) 
# validation set
validation_generator = validation_datagen.flow_from_dataframe(vdf,x_col= "Image", y_col = "Label", classes = buildingCodes, target_size=(500,500), BATCH_SIZE = 24)


Found 2794 validated image filenames belonging to 16 classes.
Found 240 validated image filenames belonging to 16 classes.


In [0]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, MaxPooling2D
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Activation
from keras import optimizers
import keras
# saving the weights 
mycall_back = keras.callbacks.ModelCheckpoint('MyModel-{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, 
                                          save_best_only=True, save_weights_only=False, mode='auto', period=1)
# model layers
image_input = Input((500,500,3))
image_model = Conv2D(64,(10,10), strides=(4,4))(image_input)
image_model = Activation('relu')(image_model)
image_model = Conv2D(128,(4,4))(image_input)
image_model = Activation('relu')(image_model)

image_model = MaxPooling2D(pool_size=(3,3))(image_model)
image_model = Conv2D(32,(3,3))(image_input)
image_model = Activation('relu')(image_model)

image_model = Conv2D(32,(3,3), strides=(2,2))(image_model)
image_model = Activation('relu')(image_model)
image_model = MaxPooling2D(pool_size=(3,3))(image_model)
image_model = Conv2D(32,(3,3), strides=(2,2))(image_model)
image_model = Activation('relu')(image_model)
image_model = Conv2D(32,(3,3), strides=(1,1))(image_model)
image_model = Activation('relu')(image_model)
image_model = MaxPooling2D(pool_size=(3,3))(image_model)
image_model = Dropout(0.5)(image_model)
image_model = Flatten()(image_model)
image_model = Dense(250)(image_model)
image_model = Activation('relu')(image_model)
#16 classes softmax function as last layer
output = Dense(16, activation='softmax')(image_model)

model = Model(inputs=image_input, outputs=output)
# adam optimizer with categorical_crossentropy
model.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 500, 500, 3)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 498, 498, 32)      896       
_________________________________________________________________
activation_3 (Activation)    (None, 498, 498, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 248, 248, 32)      9248      
_________________________________________________________________
activation_4 (Activation)    (None, 248, 248, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 82, 82, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 40, 40, 32)        9248

In [0]:

MODEL_FILE = 'model.h5'
# running model with 50 epoch and 50 steps per epoch with batch sizes of 50
history = model.fit_generator(train_generator,steps_per_epoch=50,
epochs=50,
validation_data=validation_generator,
validation_steps=10, callbacks = [mycall_back])
model.save(MODEL_FILE)




In [0]:
!wget https://github.com/rohank07/ML_A3/blob/master/model.zip?raw=true
!unzip model.zip?raw=true
# loading model from github repository
from keras.models import load_model
from google.colab import files
model = load_model('model/model.h5') # loading model

In [0]:
#testing on training set
dic = {}
for building in buildingCodes:
  y_pred = []
  y_true = []
  #preprocessing the test set
  dataframe = pd.DataFrame(test)
  predict_gen = ImageDataGenerator(
      rescale = 1./255, fill_mode='nearest',
      width_shift_range=0.2,
      height_shift_range=0.2)
  
  pred_gen = predict_gen.flow_from_dataframe(
      tdf, #test data frame
      x_col= "Image", 
      y_col = "Label", 
      classes = buildingCodes, 
      target_size=(500,500),
      batch_size = 50) 
  probs = model.predict(pred_gen)
  for i in range(15):
    j = np.argmax(probs[i]) 
    y_true.append(building) # appending the label
    y_pred.append(buildingCodes[j])

  dic[building] = [y_true,y_pred]


Found 240 validated image filenames belonging to 16 classes.
Found 240 validated image filenames belonging to 16 classes.
Found 240 validated image filenames belonging to 16 classes.
Found 240 validated image filenames belonging to 16 classes.
Found 240 validated image filenames belonging to 16 classes.
Found 240 validated image filenames belonging to 16 classes.
Found 240 validated image filenames belonging to 16 classes.
Found 240 validated image filenames belonging to 16 classes.
Found 240 validated image filenames belonging to 16 classes.
Found 240 validated image filenames belonging to 16 classes.
Found 240 validated image filenames belonging to 16 classes.
Found 240 validated image filenames belonging to 16 classes.
Found 240 validated image filenames belonging to 16 classes.
Found 240 validated image filenames belonging to 16 classes.
Found 240 validated image filenames belonging to 16 classes.
Found 240 validated image filenames belonging to 16 classes.


In [0]:
from sklearn import metrics
from sklearn.metrics import classification_report
# precision and recall scores
y_true = []
y_pred = []
scores = {}
for i in dic:
  y_true += dic[i][0]
  y_pred += dic[i][1]
# Lowest recall RB and SA
print(metrics.recall_score(y_true,y_pred, labels=buildingCodes, average =None ,sample_weight=None))
# best precision DT
print(metrics.precision_score(y_true,y_pred, labels=buildingCodes, average =None ,sample_weight=None))

[0.         0.26666667 0.         0.6        0.         0.
 0.         0.         0.         0.         0.         0.
 0.06666667 0.         0.06666667 0.        ]
[0.         0.06779661 0.         0.07258065 0.         0.
 0.         0.         0.         0.         0.         0.
 0.03030303 0.         0.0625     0.        ]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
